In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Masking
from keras.layers import Attention, LayerNormalization, Dropout
from keras.optimizers import Adam
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.chdir("/content/drive/My Drive/")
os.getcwd()

'/content/drive/My Drive'

In [5]:
df=pd.read_csv('output.csv')

In [6]:
df

,english,sinhala
0,you will receive a package in the mail,ඔයාට තැපෑලෙන් පැකේජ් එකක් හම්බවේවි
1,you will receive a confirmation code after com...,ලියාපදිංචිය සම්පූර්ණ කලාට පස්සෙ ඔයාට තහවුරු කි...
2,you will receive a discount on your next purchase,ඊලඟ මිලදී ගැනීම කරන කොට ඔයාට වට්ටමක් හම්බවේවි
3,you will receive a phone call from our custome...,ඔයාට අපේ පාරිභෝගික සේවා කණ්ඩායමෙන් දුරකථන ඇමතු...
4,you will receive a notification when your orde...,ඔයාගේ ඇණවුම සූදානම් උනාට පස්සෙ ඔයාට දැනුම් දීම...
...,...,...
80679,Deep in the enchanted forest a group of woodla...,වශීකෘත වනාන්තරයේ ගැඹුරින් නුවණැති මහලු බකමූණෙක...
80680,On a sunny summer day a group of friends gathe...,අව්ව සහිත ගිම්හාන දිනයක මිතුරන් පිරිසක් වැලි ම...
80681,In a quaint seaside village a mysterious stran...,විචිත්‍රවත් මුහුදු වෙරළේ ගම්මානයකට අද්භූත ආගන්...
80682,Thomas an aspiring writer found inspiration in...,අභිලාෂකාමී ලේඛකයෙකු වන තෝමස් සෑම අස්සක් මුල්ලක...


In [7]:
spa_sentences = df["sinhala"] + "|" + df["english"]

# Save the concatenated sentences to a text file
with open("spa.txt", "w", encoding="utf-8") as file:
    for sentence in spa_sentences:
        file.write(str(sentence) + "\n")

In [8]:
text_file ="spa.txt"

In [9]:
text_file ="spa.txt"
with open(text_file) as f:
  lines=f.read().split("\n")[:-1]

In [10]:
i=0
for line in lines:
  print(line)
  i=i+1
  if(i==20):
    break

ඔයාට තැපෑලෙන් පැකේජ් එකක් හම්බවේවි|you will receive a package in the mail
ලියාපදිංචිය සම්පූර්ණ කලාට පස්සෙ ඔයාට තහවුරු කිරීමේ කේතයක්  හම්බවේවි|you will receive a confirmation code after completing the registration
ඊලඟ මිලදී ගැනීම කරන කොට ඔයාට වට්ටමක් හම්බවේවි|you will receive a discount on your next purchase
ඔයාට අපේ පාරිභෝගික සේවා කණ්ඩායමෙන් දුරකථන ඇමතුමක් හම්බවේවි|you will receive a phone call from our customer service team
ඔයාගේ ඇණවුම සූදානම් උනාට පස්සෙ ඔයාට දැනුම් දීමක් හම්බවේවි|you will receive a notification when your order is ready for pickup
ඔයාට පැය 24ක් ඇතුලත ඔයාගෙ විමසීමට පිළිතුරක්  හම්බවේවි|you will receive a response to your inquiry within 24 hours
ඔයාට ඔයාගේ පක්ෂපාතිත්වය වෙනුවෙන් තෑග්ගක් හම්බවේවි|you will receive a gift for your loyalty
ඔයාට උත්සවයට ආරාධනාවක් හම්බවේවි|you will receive an invitation to the event
ඔයාට රිටන් කරන භාණ්ඩය වෙනුවෙන් ආපසු මුදල් ගෙවීමක් හම්බවේවි|you will receive a refund for the returned item
ඔයාට විස්තර එක්ක  ඊමේල් එකක් හම්බවේවි|you'll receive an e

In [11]:
for x in range(len(lines)-10,len(lines)):
  print(lines[x])

එක් කලෙක කුඩා ගමක ලිලී නම් තරුණියක් ජීවත් වූ අතර ඇයට ලස්සනම මල් වගා කිරීම සඳහා විශේෂ තෑග්ගක් තිබුණි|Once upon a time in a small village there lived a young girl named Lily who had a special gift for growing the most beautiful flowers
දුර රටක ශ්‍රීමත් ආතර් නම් නිර්භීත නයිට්වරයා නපුරු මායාකාරයෙකුගෙන් පැහැරගෙන ගිය කුමරියක් බේරා ගැනීමට භයානක ගමනක් ආරම්භ කළේය|In a distant land a brave knight named Sir Arthur embarked on a perilous journey to rescue a kidnapped princess from an evil sorcerer
කුතුහලය දනවන ගවේෂකයෙකු වන එමිලි ඇමසන් වනාන්තරයේ ගැඹුරින් ගිලිහී ගිය පැරණි නගරයක සැඟවුණු වස්තු සොයා ගැනීමට ත්‍රාසජනක ගමනක් ආරම්භ කළාය|Emily a curious explorer set out on an adventure to uncover the hidden treasures of an ancient lost city deep in the Amazon rainforest
සාමකාමී නගරයක් වන විලෝබෲක්හි ඔලිවර් නම් දඟකාර බළලාට විනෝදජනක සහ අනපේක්ෂිත දුෂ්කරතාවන්ට පත්වීමේ දක්ෂතාවයක් තිබුණි|In the peaceful town of Willowbrook a mischievous cat named Oliver had a talent for getting into amusing and unexpected predicam

In [28]:
import random
import string
import re

text_pairs = []


for line in lines:
    if '|' in line:
        english, sinhala = line.split("|")
        sinhala = "[start]" + sinhala.strip() + "[end]"
        text_pairs.append((english.strip(), sinhala))
for i in range(3):
    print(random.choice(text_pairs))


('bike', '[start]බයිසිකලය[end]')
('tool', '[start]මෙවලම[end]')
('We eat dinner early, and then we go for a walk.', '[start]අපි රෑ කෑම වේලාසනින් කනවා, ඊට පස්සේ අපි ඇවිදින්න යනවා.[end]')


In [29]:
import random
random.shuffle(text_pairs)

In [30]:
num_val_samples =int(0.15*len(text_pairs))
num_train_samples=len(text_pairs) -2* num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [31]:
print("Total sentences:",len(text_pairs))
print("Training set size:",len(train_pairs))
print("Validation set size:",len(val_pairs))
print("Testing set size:",len(test_pairs))

Total sentences: 428
Training set size: 300
Validation set size: 64
Testing set size: 64


In [32]:
len(train_pairs)+len(val_pairs)+len(test_pairs)

428

In [33]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

In [34]:
f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~¿]'

In [35]:
from tensorflow import keras
from tensorflow.keras import layers
def custom_standardization(input_string):
  lowercase = tf.strings.lower(input_string)
  return tf.strings.regex_replace(
    lowercase, f"[{re.escape(strip_chars)}]", "")
vocab_size = 15000
sequence_length = 20
source_vectorization = layers.TextVectorization(
  max_tokens=vocab_size,
  output_mode="int",
  output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
  max_tokens=vocab_size,
  output_mode="int",
  output_sequence_length=sequence_length + 1,
  standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]

In [39]:
import tensorflow as tf
from tensorflow.keras import layers
import re

# Define your custom standardization function
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")


# Adapt the vectorization layers
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)


In [41]:
for inputs, targets in train_ds.take(1):
  print(f"inputs['english'].shape:{inputs['english'].shape}")
  print(f"inputs['sinhala'].shape:{inputs['sinhala'].shape}")
  print(f"targets.shape:{targets.shape}")
print(len(list(train_ds.as_numpy_iterator())))


inputs['english'].shape:(44, 20)
inputs['sinhala'].shape:(44, 20)
targets.shape:(44, 20)
5


In [42]:

import string
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [43]:
import tensorflow as tf
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config


In [44]:
import tensorflow as tf
from tensorflow.keras import layers

class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask
        )
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)


In [45]:
import tensorflow as tf
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


In [46]:
embed_dim = 256
dense_dim = 2048
num_heads = 8


# Define the encoder inputs
encoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="english")

# Add positional embedding to the encoder inputs
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

# Encode the inputs using TransformerEncoder
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

# Define the decoder inputs
decoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="sinhala")

# Add positional embedding to the decoder inputs
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)

# Decode the inputs using TransformerDecoder
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)

# Apply dropout
x = layers.Dropout(0.5)(x)

# Generate decoder outputs
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

# Create the transformer model
transformer = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 sinhala (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding (Posi  (None, None, 256)            3845120   ['english[0][0]']             
 tionalEmbedding)                                                                                 
                                                                                                  
 positional_embedding_1 (Po  (None, None, 256)            3845120   ['sinhala[0][0]']         

In [47]:
transformer.compile(optimizer="rmsprop",
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])

transformer.fit(train_ds, epochs=30, validation_data=val_ds)


Epoch 1/30
5/5 [==============================] - 11s 362ms/step - loss: 7.8507 - accuracy: 0.5193 - val_loss: 6.7850 - val_accuracy: 0.5614
Epoch 2/30
5/5 [==============================] - 0s 80ms/step - loss: 5.9681 - accuracy: 0.6085 - val_loss: 5.6620 - val_accuracy: 0.5614
Epoch 3/30
5/5 [==============================] - 0s 69ms/step - loss: 4.7688 - accuracy: 0.6085 - val_loss: 4.9248 - val_accuracy: 0.5614
Epoch 4/30
5/5 [==============================] - 0s 70ms/step - loss: 4.0334 - accuracy: 0.6085 - val_loss: 4.6403 - val_accuracy: 0.5614
Epoch 5/30
5/5 [==============================] - 0s 69ms/step - loss: 3.8601 - accuracy: 0.5984 - val_loss: 4.5711 - val_accuracy: 0.5614
Epoch 6/30
5/5 [==============================] - 0s 79ms/step - loss: 3.5325 - accuracy: 0.6085 - val_loss: 4.5452 - val_accuracy: 0.5614
Epoch 7/30
5/5 [==============================] - 0s 90ms/step - loss: 3.3866 - accuracy: 0.6085 - val_loss: 4.4866 - val_accuracy: 0.5614
Epoch 8/30
5/5 [=========

In [48]:
import numpy as np

# Define the vocabulary and index lookup for Spanish
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break

    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]

for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))


-
helicopter
[start]                    
-
worth
[start]                    
-
trip
[start]                    
-
joy
[start]                    
-
relationship
[start]                    
-
brother
[start]                    
-
big
[start]                    
-
then
[start]                    
-
twenty
[start]                    
-
trip
[start]                    
-
king
[start]                    
-
camera
[start]                    
-
then
[start]                    
-
laptop
[start]                    
-
two
[start]                    
-
pen
[start]                    
-
tailor
[start]                    
-
helicopter
[start]                    
-
armor
[start]                    
-
security
[start]                    


In [49]:
import numpy as np

# Define the vocabulary and index lookup for Sinhala
sin_vocab = target_vectorization.get_vocabulary()
sin_index_lookup = dict(zip(range(len(sin_vocab)), sin_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = sin_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break

    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]

for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print("English: ", input_sentence)
    print("Sinhala: ", decode_sequence(input_sentence))


-
English:  helicopter
Sinhala:  [start]                    
-
English:  doubt
Sinhala:  [start]                    
-
English:  friendship
Sinhala:  [start]                    
-
English:  fall
Sinhala:  [start]                    
-
English:  war
Sinhala:  [start]                    
-
English:  then
Sinhala:  [start]                    
-
English:  star
Sinhala:  [start]                    
-
English:  grandfather
Sinhala:  [start]                    
-
English:  uncertainty
Sinhala:  [start]                    
-
English:  challenge
Sinhala:  [start]                    
-
English:  relationship
Sinhala:  [start]                    
-
English:  one
Sinhala:  [start]                    
-
English:  While the dog runs in the park, the cat sleeps peacefully at home.
Sinhala:  [start]                    
-
English:  king
Sinhala:  [start]                    
-
English:  out
Sinhala:  [start]                    
-
English:  match
Sinhala:  [start]                    
-
English:  pen
Sinh

In [51]:
import numpy as np

# Define the vocabulary and index lookup for Sinhala
sin_vocab = target_vectorization.get_vocabulary()
sin_index_lookup = dict(zip(range(len(sin_vocab)), sin_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = sin_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break

    return decoded_sentence

# Get English input from the user
input_sentence = input("Enter an English sentence: ")

# Translate the input sentence to Sinhala
translated_sentence = decode_sequence(input_sentence)

# Print the translated sentence
print("Sinhala translation:", translated_sentence)


Enter an English sentence: How are you?
Sinhala translation: [start]                    
